# Lab 12-3: GPT2 Model for Language Understanding
## Exercise: Building a Korean Chatbot
This exercise is taken from Github Storage for "What is Natural Language Processing?" by Wonjoon Yu.<br>
https://github.com/ukairia777/tensorflow-nlp-tutorial

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 29.0 MB/s 
     |████████████████████████████████| 6.6 MB 61.0 MB/s 
     |████████████████████████████████| 596 kB 68.5 MB/s 
     |████████████████████████████████| 101 kB 11.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import tensorflow as tf
import tqdm
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

The GPT2 Model transformer for TensorFlow with a language modeling head on top (linear layer with weights tied to the input embeddings).

If you choose this second option, there are three possibilities you can use to gather all the input Tensors in the first positional argument :

a single Tensor with input_ids only and nothing else: `model(inputs_ids)`

a list of varying length with one or several input Tensors IN THE ORDER given in the docstring: `model([input_ids, attention_mask])` or `model([input_ids, attention_mask, token_type_ids])`

a dictionary with one or several input Tensors associated to the input names given in the docstring: `model({'input_ids': input_ids, 'token_type_ids': token_type_ids})`

https://huggingface.co/transformers/v3.0.2/index.html

In [4]:
### START CODE HERE ###

# find & assign tokenizer and model; 'skt/kogpt2-base-v2'

tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

### END CODE HERE ###

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.8.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.7.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [5]:
model.summary()

Model: "tfgpt2lm_head_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 125164032 
 r)                                                              
                                                                 
Total params: 125,164,032
Trainable params: 125,164,032
Non-trainable params: 0
_________________________________________________________________


In [6]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-' * 10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

1
1
3
----------
</s>
<usr>
<pad>
<sys>


In [7]:
import pandas as pd
import urllib.request

Import a Korean chatbot dataset made by songys: <br>
https://github.com/songys/Chatbot_data <br>
To find more Korean dataset, check this site: <br>
https://github.com/ko-nlp/Korpora

In [8]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", 
                           filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')

In [9]:
display(train_data)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [10]:
batch_size = 32

In [11]:
def get_chat_data():
    for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
        bos_token = [tokenizer.bos_token_id]
        eos_token = [tokenizer.eos_token_id]
        sent = tokenizer.encode('<usr>' + question + '<sys>' + answer) 
        yield bos_token + sent + eos_token

In [12]:
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

In [13]:
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [14]:
for batch in dataset:
    print(batch[0])
    break

tf.Tensor(
[    1     2  9349  7888   739  7318   376     4 12557  6824  9108  9028
  7098 25856     1     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3], shape=(30,), dtype=int32)


In [15]:
tokenizer.decode(batch[0])

'</s><usr> 12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [16]:
print(tokenizer.encode('</s><usr> 12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'))

[1, 2, 9349, 7888, 739, 7318, 376, 4, 12557, 6824, 9108, 9028, 7098, 25856, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [17]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [18]:
steps = len(train_data) // batch_size + 1
print(steps)

370


In standard text generation fine-tuning, since we are predicting the next token given the text we have seen thus far, the labels are just the shifted encoded tokenized input. However, GPT's CLM (causal language model) uses look-ahead masks to hide the next tokens, which has the same effect as the labels are automatically shifted inside the model. Therefore, we can set as `labels=input_ids`.

In [19]:
EPOCHS = 3

for epoch in range(EPOCHS):
    epoch_loss = 0
  
    for batch in tqdm.notebook.tqdm(dataset, total=steps):
        with tf.GradientTape() as tape:
            result = model(batch, labels=batch)
            loss = result[0]
            batch_loss = tf.reduce_mean(loss)
            
        grads = tape.gradient(batch_loss, model.trainable_variables)
        adam.apply_gradients(zip(grads, model.trainable_variables))
        epoch_loss += batch_loss / steps
  
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    1] cost = 2.12708116


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    2] cost = 1.69853389


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    3] cost = 1.37692618


In [20]:
text = '오늘도 좋은 하루!'

In [21]:
sent = '<usr>' + text + '<sys>'

In [22]:
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])

In [23]:
output = model.generate(input_ids, max_length=50, early_stopping=True, eos_token_id=tokenizer.eos_token_id)

In [24]:
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())
print(decoded_sentence)

</s><usr> 오늘도 좋은 하루!<sys> 오늘도 좋은 하루네요.</s>


In [25]:
decoded_sentence.split('<sys> ')[1].replace('</s>', '')

'오늘도 좋은 하루네요.'

In [26]:
output = model.generate(input_ids, max_length=50, do_sample=True, top_k=10)
tokenizer.decode(output[0].numpy().tolist())

'</s><usr> 오늘도 좋은 하루!<sys> 좋은 시간 보내고 오세요.</s>'

In [27]:
def return_answer_by_chatbot(user_text):
  sent = '<usr>' + user_text + '<sys>'
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
  input_ids = tf.convert_to_tensor([input_ids])
  output = model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
  return chatbot_response

In [28]:
return_answer_by_chatbot('안녕! 반가워~')

'반가운데 짝사랑인지 확인해보세요.'

In [29]:
return_answer_by_chatbot('너는 누구야?')

'먼저 물어봐보세요.'

In [30]:
return_answer_by_chatbot('나랑 영화보자')

'영화와 친해지는 방법 좀 알려주세요.'

In [31]:
return_answer_by_chatbot('너무 심심한데 나랑 놀자')

'심경의 변화가 있었나 봐요.'

In [32]:
return_answer_by_chatbot('영화 해리포터 재밌어?')

'재미로 봤는데 영화보셨나봐요.'

In [33]:
return_answer_by_chatbot('너 딥 러닝 잘해?')

'개인 데스크톱을 해보세요.'

In [38]:
return_answer_by_chatbot('이름이 뭐예요?')

'저랑 연락이 가능한 친구한테 연락하는 거요.'